# N-gram Text Correction using Natural Language Processing

## By Brea Koenes

---

### Overview

There is a physical manuscript, possibly by William Shakespeare, with parts lost due to time and poor storage. Using Natural Language Processing, deploy n-gram and fivegram models to predict and weave in the missing words, aiming to restore the manuscript.

Datasets:  

- WS_train.txt - All WS works.

- WS_test.txt - The manuscript, with the words lost marked as `<DELETED>`.
    
- WS_validation - Text to validade models performance.
    
    
**Objective**: Implement an N-gram language model that can predict missing words in a text corpus, using the works of William Shakespeare as a basis for model training and evaluation.

**Tasks:**

1. **Data Preparation**: Load and preprocess the training and test texts, preparing them for the N-gram model training.
2. **N-gram Model Training - parts A and B**: Utilize the `WS_train.txt` to create an N-gram language model that learns the word sequences and their probabilities.
3. **Text Correction**: Apply the trained N-gram model to predict and fill in the `<DELETED>` placeholders in `WS_test.txt`.
4. **Evaluation**: Compare the corrected text against `WS_validation.txt` to evaluate the accuracy of the N-gram model's predictions.

### 1 - Data Preparation

Define specific functions for loading, cleaning, generating N-grams, and building a vocabulary from the text data.

#### 1.1 - Import necessary libraries:

In [27]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams

#### 1.2 - Load the Text

Create a function `load_text` that reads a text file and returns its contents as a string.

In [28]:
# Load file
def load_text(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()

train_text = load_text('WS_train.txt')

#### 1.3 - Clean the Text

Define a function named `clean_text` to standardize, tokenize, and remove punctuation from the text data, while retaining the **<DELETED>** placeholders. 

The function will:
    
1. Convert the text to lowercase.
2. Tokenize
3. Remove all punctuation tokens using `string.punctuation`.
4. Remove stop words tokens using NLTK's `stopwords.words('english')`


In [29]:
# More imports
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [30]:
# Takes list of text as input and returns a list of cleaned tokens.
def clean_text(texts):
    
    # Lowercase
    texts = texts.lower()

    # Tokenize
    tokens = word_tokenize(texts)
    
    # Initialize
    punctuation_list = list(string.punctuation)
    stop_words = stopwords.words('english')
    
    cleaned_tokens = []
    for token in tokens:
        if token == "deleted":
            cleaned_tokens.append(token) # Keep if it's <deleted>
        elif token not in punctuation_list and token not in stop_words: 
            cleaned_tokens.append(token) # Otherwise also keep if not punctuation or stop word 

    # Replace tokenized strings in list to return it to original <DELETED>
    old_substring = "deleted"  
    new_substring = "<DELETED>"  
 
    cleaned_tokens = [s.replace(old_substring, new_substring) for s in cleaned_tokens]  

    return cleaned_tokens

# Apply to list
cleaned_train_text = clean_text(train_text)
cleaned_train_text[0:30]

['1609',
 'sonnets',
 'william',
 'shakespeare',
 '1',
 'fairest',
 'creatures',
 'desire',
 'increase',
 'thereby',
 'beauty',
 "'s",
 'rose',
 'might',
 'never',
 'die',
 'riper',
 'time',
 'decease',
 'tender',
 'heir',
 'might',
 'bear',
 'memory',
 'thou',
 'contracted',
 'thine',
 'bright',
 'eyes',
 "feed'st"]

#### 1.4 - Create N-grams

Define a function `create_ngrams` that will convert a list of tokens into a list of N-grams. The function should take the following parameters:

- `tokens`: A list of words (tokens) from which to create N-gram

- `n`: The order of the N-gram (e.g., 2 for bigrams, 3 for trigrams, etc.

In [31]:
from nltk.util import ngrams

def create_ngrams(tokens, n):
    
    modified_tokens = ['<s>'] * (n-1) + tokens +['</s>']

    n_grams = list(ngrams(modified_tokens,n))

    return n_grams

# Apply to data
train_bigrams = create_ngrams(cleaned_train_text, 2)
train_fivegrams = create_ngrams(cleaned_train_text, 5)
train_fivegrams[0:15]

[('<s>', '<s>', '<s>', '<s>', '1609'),
 ('<s>', '<s>', '<s>', '1609', 'sonnets'),
 ('<s>', '<s>', '1609', 'sonnets', 'william'),
 ('<s>', '1609', 'sonnets', 'william', 'shakespeare'),
 ('1609', 'sonnets', 'william', 'shakespeare', '1'),
 ('sonnets', 'william', 'shakespeare', '1', 'fairest'),
 ('william', 'shakespeare', '1', 'fairest', 'creatures'),
 ('shakespeare', '1', 'fairest', 'creatures', 'desire'),
 ('1', 'fairest', 'creatures', 'desire', 'increase'),
 ('fairest', 'creatures', 'desire', 'increase', 'thereby'),
 ('creatures', 'desire', 'increase', 'thereby', 'beauty'),
 ('desire', 'increase', 'thereby', 'beauty', "'s"),
 ('increase', 'thereby', 'beauty', "'s", 'rose'),
 ('thereby', 'beauty', "'s", 'rose', 'might'),
 ('beauty', "'s", 'rose', 'might', 'never')]

In [32]:
train_fivegrams[-15:]

[('distributed', 'used', 'commercially', 'prohibited', 'commercial'),
 ('used', 'commercially', 'prohibited', 'commercial', 'distribution'),
 ('commercially', 'prohibited', 'commercial', 'distribution', 'includes'),
 ('prohibited', 'commercial', 'distribution', 'includes', 'service'),
 ('commercial', 'distribution', 'includes', 'service', 'charges'),
 ('distribution', 'includes', 'service', 'charges', 'download'),
 ('includes', 'service', 'charges', 'download', 'time'),
 ('service', 'charges', 'download', 'time', 'membership.'),
 ('charges', 'download', 'time', 'membership.', 'end'),
 ('download', 'time', 'membership.', 'end', 'etext'),
 ('time', 'membership.', 'end', 'etext', 'complete'),
 ('membership.', 'end', 'etext', 'complete', 'works'),
 ('end', 'etext', 'complete', 'works', 'william'),
 ('etext', 'complete', 'works', 'william', 'shakespeare'),
 ('complete', 'works', 'william', 'shakespeare', '</s>')]

#### 1.5 - Build Vocabulary:

Define a function `build_vocab` that creates a set of unique words from a list of tokens, EXCLUDING the `<DELETED>` placeholder. The function takes the following parameter:

- `tokens`: A **list** of clean tokens from which to build the vocabulary.
    
The function `build_vocab` should return a `set` of unique tokens, which is the vocabulary.

In [33]:
# Create vocab set
def build_vocab(tokens):
    vocab_set = {token for token in tokens if token != '<DELETED>'}
    return vocab_set

# Apply to data
vocab = build_vocab(cleaned_train_text)

### 2A - N-gram Model Training - part A

Training the N-gram model. This process involves calculating the frequency distribution of N-grams and estimating their probabilities based on the training corpus. These statistics will then be used to predict the next word in a sequence or to determine the most likely correction in a text.


#### 2A.1 - Import necessary libraries:

>```python
from nltk import FreqDist, ConditionalFreqDist, ConditionalProbDist, MLEProbDist

In [34]:
from nltk import FreqDist, ConditionalFreqDist, ConditionalProbDist, MLEProbDist

#### 2A.2 - Calculate Frequency Distribution

Create a function `calculate_ngram_freq` to calculate the frequency distribution of N-grams in the training data, employing the `FreqDist` class from the `nltk` library. The function should take the following parameter:

- `ngrams_list`: A list of N-grams for which to calculate the frequency distribution.

The function should return:

- A `FreqDist` object representing the Frequency Distribution of the input N-grams.

Using the function `calculate_ngram_freq`, create two variables named `bigram_freq_dist` and `fivegram_freq_dist`. Use the appropriate N-grams list.

In [35]:
def calculate_ngram_freq(ngrams_list):
    freq_dist = FreqDist(tuple(ngram) for ngram in ngrams_list)
    return freq_dist

# Apply to data
bigram_freq_dist = calculate_ngram_freq(train_bigrams)
fivegram_freq_dist = calculate_ngram_freq(train_fivegrams)

#### 2A.3 - Probability Estimation

Create a function `estimate_ngram_probabilities` to estimate the conditional probabilities of N-grams, utilizing the `ConditionalFreqDist` and `ConditionalProbDist` classes along with a probability distribution such as `MLEProbDist` from the `nltk.probability` module. The function should take the following parameter:
    
- `ngrams_list`: A list of N-grams for which to estimate conditional probabilities.
   
The function should return:

- A `ConditionalProbDist` object representing the conditional probabilities of the input N-grams.


Using the function `Probability Estimation `, create two variables named `bigram_prob_dist` and `fivegram_prob_dist`. Use the appropriate N-grams list.

In [36]:
from nltk.probability import ConditionalFreqDist, ConditionalProbDist, MLEProbDist

def estimate_ngram_probabilities(ngrams_list):
    cfdist = ConditionalFreqDist()
    for ngram in ngrams_list:
        condition = tuple(ngram[:-1])
        word = ngram[-1]
        cfdist[condition][word] += 1
    
    return ConditionalProbDist(cfdist, MLEProbDist)

# Estimate probabilities
bigram_prob_dist = estimate_ngram_probabilities(train_bigrams)
fivegram_prob_dist = estimate_ngram_probabilities(train_fivegrams)

### 2B - N-gram Model Training - part B

Create the core function of the ngram NLP: the function `predict_next_word`.
    

#### 2A.1 - Import necessary libraries:

>```python
from nltk.probability import ConditionalProbDist

In [37]:
from nltk.probability import ConditionalProbDist

#### 2A.2 - Predict next word

Create a function `predict_next_word` that utilizes the conditional probabilities to predict the most probable next word after a given context.

The function should take the following parameters:

- A context, which is a tuple of words that precedes the word to be predicted. The size of the context should be N-1 for an N-gram model.
- A `ConditionalProbDist` object that has been previously computed from the training data.
- Optionally accepts an integer `top_n` that specifies the number of top probable next words to return (default is 1, which returns the most probable next word).


The function should handle cases where the context is not found in the `ConditionalProbDist`, returning the default value `<UNK>`.

In [38]:
# Predict next word function
def predict_next_word(context, prob_dist, top_n=1):
    if context in prob_dist:
        return prob_dist[context].max() if top_n == 1 else prob_dist[context].samples()[:top_n]
    return '<UNK>'

### 3 - Text Correction

After training our N-gram model, the next step is to apply it to correct texts that contain placeholders indicating missing words. In this section, import the test text and use our model to predict the words that should replace the `<DELETED>` placeholders.


#### 3.1 - Correction Function

Create a function `correct_text_with_ngrams` that searches for `<DELETED>` placeholders in the test data and uses the `predict_next_word` function to find the most probable replacement.

The function should take the following parameters:

- `text_data`: The list of tokens from the test data, including `<DELETED>` placeholders.
- `ngram_model`: The trained N-gram model to use for prediction (e.g., bigram or fivegram model).
- `n`: The order of the N-gram (e.g., 2 for bigrams, 3 for trigrams, etc.).

The function should return:

- `corrected_text`: A **list** of tokens where `<DELETED>` placeholders have been replaced with the most probable word predicted by the model.

In [39]:
# Text correction function
def correct_text_with_ngrams(text_data, ngram_model, n):
    corrected_text = []
    for i in range(len(text_data)):
        if text_data[i] == '<DELETED>':
            context = tuple(corrected_text[-(n-1):])
            predicted_word = predict_next_word(context, ngram_model)
            corrected_text.append(predicted_word)
        else:
            corrected_text.append(text_data[i])
    return corrected_text

#### 3.2 - Load and Clean Test Data

Import the `WS_test.txt` file using the `load_text` function. Then, apply the `clean_text` function to prepare the data for correction.

Execute the `load_text` function to import the content of `WS_test.txt` and store it in a variable named `test_text`.
Apply `clean_text` to `test_text` to obtain a tokenized and cleaned list of words, including `<DELETED>` placeholders, and store it in a variable named `cleaned_test_text`.

In [40]:
# Read in and apply clean_text
test_text = load_text('WS_test.txt')
cleaned_test_text = clean_text(test_text)

#### 3.3 Applying Bigram Model

Apply the `correct_text_with_ngrams` function to the `cleaned_test_text` using the bigram `bigram_prob_dist` object created before. Save the output to a variable named `corrected_test_text_bigram`.

In [41]:
corrected_test_text_bigram = correct_text_with_ngrams(cleaned_test_text, bigram_prob_dist, n = 2)

#### 3.4 Applying Fivegram Model

Apply the `correct_text_with_ngrams` function to the `cleaned_test_text` using the bigram `fivegram_prob_dist` object created before. Save the output to a variable named `corrected_test_text_fivegram`.

In [42]:
corrected_test_text_fivegram = correct_text_with_ngrams(cleaned_test_text, fivegram_prob_dist, n = 5)

### Evaluation

Evaluate the performance of our N-gram model. Calculate the accuracy of our model by comparing the predicted text against a validation text that contains the correct words.

#### Objectives:

1. **Import and Clean Validation Data**: Load and preprocess the `WS_validation.txt` file to obtain a clean list of tokens for accuracy comparison.

2. **Accuracy Calculation**: Use the supplied function `calculate_accuracy` to calculate the accurary for both models.

#### Tasks:

1. **Load Validation Text**: Use the `load_text` function to import the `WS_validation.txt` file.

2. **Clean Validation Text**: Apply the `clean_text` function to the imported validation text to produce a list of clean tokens for comparison.

3. **Accuracy Calculation**: This function takes:

   - `test_tokens`: A list of tokes with the `<DELETED>` placeholder (before correction).
   - `corrected_tokens`: A list of tokens that have been corrected by the N-gram model (either bigram or fivegram).
   - `validation_tokens`: A list of clean tokens from the validation text.<B></B>
   
The function should return the accuracy as a float, calculated as the number of correct predictions divided by the total number of predictions.

#### Steps:

1. **Import and Clean Validation Data**:

   - Use the `load_text` function to load the `WS_validation.txt` as the validation text data. Store it as `validation_text`.
   - Apply the `clean_text` function to the loaded `validation_text` to obtain a list of tokens for accuracy comparison, named `cleaned_validation_text`.<B></B>

2. **Accuracy Calculation**:

   - Store the resulting accuracy scores in variables named `bigram_accuracy` and `fivegram_accuracy`.<B></B>




In [43]:
def calculate_accuracy(test_tokens, corrected_tokens, validation_tokens):
    if (len(test_tokens) != len(corrected_tokens)) or (len(test_tokens) != len(validation_tokens)):
        print("Test Tokens, Validation Token and Corrected Tokens must have the same length")
        return

    correct_predictions = 0
    all_predictions = 0
    for i in range(len(test_tokens)):
        if test_tokens[i] == '<DELETED>':
            all_predictions+=1
            if corrected_tokens[i] == validation_tokens[i]:
                   correct_predictions+=1

    return correct_predictions/all_predictions

In [44]:
# Read in data and clean text
validation_text = load_text('WS_validation.txt')
cleaned_validation_text = clean_text(validation_text)

In [45]:
# Evaluate model
bigram_accuracy = calculate_accuracy(cleaned_test_text, corrected_test_text_bigram, cleaned_validation_text)
fivegram_accuracy = calculate_accuracy(cleaned_test_text, corrected_test_text_fivegram, cleaned_validation_text)

### Export Models

In [46]:
import pickle

with open('fivegram_prob_dist.pkl', 'wb') as file:
    pickle.dump(fivegram_prob_dist , file)